In [ ]:
import pandas as pd

# Sample dataframe
data = {
    'O/S Cash USD': [100, 200, 150, 300, 400],
    'Cpty Name': ['Company A', 'Company B', 'Company A', 'Company C', 'Company B'],
    'Treated_Comments': ['Comment1', 'Comment2', 'Comment3', 'Comment4', 'Comment5'],
    'Treated_Fails_code': ['Code1', 'Code2', 'Code1', 'Code2', 'Code1']
}

df = pd.DataFrame(data)

# Group by 'Treated_Fails_code' and 'Cpty Name', then count occurrences
grouped = df.groupby(['Treated_Fails_code', 'Cpty Name']).size().reset_index(name='Count')

# Sort by 'Treated_Fails_code' and 'Count' in descending order
sorted_grouped = grouped.sort_values(by=['Treated_Fails_code', 'Count'], ascending=[True, False])

# Save the result to an Excel file
output_file = 'fails_code_counts.xlsx'
with pd.ExcelWriter(output_file) as writer:
    sorted_grouped.to_excel(writer, sheet_name='Fails_Code_Counts', index=False)

print(f"Data saved to {output_file}")


In [ ]:
import pandas as pd
import re

# Sample dataframe
data = {
    'O/S Cash USD': [100, 200, 150, 300, 400],
    'Cpty Name': ['Company A', 'Company B', 'Company A', 'Company C', 'Company B'],
    'Treated_Comments': [
        'Short to Deliver due to pending receipts from CP x',
        'Short to Deliver due to failing receipts from Cp b',
        'Cp a was short to deliver',
        'Firm was Short due to failing receipts from Cp b',
        'Cp a was short to deliver'
    ],
    'Treated_Fails_code': ['FTD', 'FTD', 'FTD', 'FTD', 'Other']
}

df = pd.DataFrame(data)

# Filter rows where 'Treated_Fails_code' is 'FTD'
ftd_df = df[df['Treated_Fails_code'] == 'FTD'].copy()

# Function to extract firm mentioned after 'from' or find other cases of failure to deliver
def extract_firm(comment):
    # Regular expression to find the firm after 'from'
    match = re.search(r'from\s([\w\s]+)', comment, re.IGNORECASE)
    if match:
        return match.group(1).strip()  # Return firm name after 'from'
    # Handle other patterns like 'Cp a was short to deliver'
    match_alt = re.search(r'([\w\s]+)\swas short to deliver', comment, re.IGNORECASE)
    if match_alt:
        return match_alt.group(1).strip()  # Return firm name before 'was short to deliver'
    return None  # If no match is found, return None

# Apply the extraction function to the 'Treated_Comments' column
ftd_df['Because of Which firm'] = ftd_df['Treated_Comments'].apply(extract_firm)

# Count the occurrences of FTD for each 'Cpty Name'
ftd_summary = ftd_df.groupby(['Cpty Name', 'Because of Which firm']).size().reset_index(name='Count')

# Save the result to the existing Excel file
output_file = 'fails_code_counts.xlsx'
with pd.ExcelWriter(output_file, mode='a', engine='openpyxl') as writer:
    # Save the new analysis in a new sheet
    ftd_summary.to_excel(writer, sheet_name='FTD_Analysis', index=False)

print(f"New analysis saved to {output_file}")


In [2]:
import xml.etree.ElementTree as ET

def add_name_tags_to_gpx(file_path, output_path):
    # Parse the GPX file
    tree = ET.parse(file_path)
    root = tree.getroot()

    # Define namespaces used in GPX file
    ns = {'gpx': 'http://www.topografix.com/GPX/1/1'}

    # Iterate over all waypoints (wpt elements)
    for wpt in root.findall('gpx:wpt', ns):
        # Check if the waypoint has a 'name' child element
        name_tag = wpt.find('gpx:name', ns)
        
        # If no name tag exists, add an empty <name> tag
        if name_tag is None:
            ET.SubElement(wpt, '{http://www.topografix.com/GPX/1/1}name').text = ''

    # Write the modified tree back to an output file
    tree.write(output_path, encoding='utf-8', xml_declaration=True)

# Example usage
file_path = r'C:\Users\shara\Downloads\\28-new-york-city-1-new-york-united-states.gpx'  # Path to the input GPX file
output_path = 'output.gpx'  # Path to save the modified GPX file

add_name_tags_to_gpx(file_path, output_path)
print(f"Modified GPX saved to {output_path}")


Modified GPX saved to output.gpx


In [1]:
import xml.etree.ElementTree as ET

def add_name_tags_to_gpx(file_path, output_path):
    # Parse the GPX file
    tree = ET.parse(file_path)
    root = tree.getroot()

    # Iterate over all <wpt> elements (waypoints)
    for wpt in root.findall('.//wpt'):
        # Check if <name> tag exists within the waypoint
        name_tag = wpt.find('name')
        
        # If <name> tag is missing, create an empty <name> tag
        if name_tag is None:
            name_element = ET.Element('name')
            name_element.text = ' '  # Add a space inside the <name> tag
            wpt.append(name_element)  # Append the new <name> tag to <wpt>

    # Write the modified tree back to an output file
    tree.write(output_path, encoding='utf-8', xml_declaration=True)

# Example usage
fname = '20-kyoto-1-honshu-japan.gpx'
file_path = f'C:/Users/shara/Downloads/{fname}'  # Path to the input GPX file
output_path = fname  # Path to save the modified GPX file

add_name_tags_to_gpx(file_path, output_path)
print(f"Modified GPX saved to {output_path}")


Modified GPX saved to 20-kyoto-1-honshu-japan.gpx


Analyzing AUM (Assets Under Management) vs. Trading Volume is important for evaluating the characteristics, liquidity, and stability of an investment vehicle, especially in the context of exchange-traded funds (ETFs) like IWM (iShares Russell 2000 ETF) or mutual funds. Here's why it's significant to consider both:

1. Liquidity
AUM reflects the total value of assets held by a fund. Larger AUM generally means the fund has more capital invested in its underlying assets, which can indicate higher liquidity and easier execution of trades without significantly impacting the price.
Trading Volume represents how many shares of the fund are traded in the market during a given time period (usually daily volume). High trading volume ensures better liquidity because it means there are more buyers and sellers, allowing investors to enter and exit positions with less price impact.
Why it's important: A fund with high AUM but low trading volume might indicate that the fund holds a lot of assets, but there may not be enough daily activity in the market for investors to easily buy or sell shares. On the other hand, high trading volume without significant AUM could signal speculative activity or short-term trading, which might not reflect the long-term strength or stability of the fund.

2. Market Impact
AUM indicates the scale of a fund’s holdings. Large AUM in an ETF implies that the fund holds a significant portion of the underlying assets. If such a fund starts making large trades (due to inflows or outflows), it could potentially impact the market for those assets.
Trading Volume helps investors gauge how much influence market trades have on the fund's price. If trading volume is low, large buy or sell orders can lead to higher volatility and price swings. However, if volume is high, these trades might have little impact.
Why it's important: A mismatch between AUM and trading volume can impact how easily an investor can execute large trades. A fund with high AUM and low trading volume could be hard to exit without causing price disruption.

3. Tracking Efficiency
AUM can influence how well an ETF tracks its benchmark index. Funds with higher AUM typically have better access to resources for managing and balancing the portfolio, which allows them to track their index more accurately.
Trading Volume affects the pricing of the ETF relative to its Net Asset Value (NAV). If trading volume is low, the ETF’s market price might deviate from the value of its underlying assets (NAV), leading to pricing inefficiencies.
Why it's important: A high-AUM, high-trading-volume ETF tends to offer better tracking of its benchmark index and provide tighter spreads between the ETF price and NAV, making it more efficient for both long-term and short-term investors.

4. Cost Efficiency
AUM is often linked to the fund's expense ratio. Larger funds can spread operational costs over more assets, which might result in lower expense ratios and make the fund more cost-effective for investors.
Trading Volume affects transaction costs for buying and selling shares. Higher volume generally leads to narrower bid-ask spreads, meaning lower costs for traders who want to move in and out of positions.
Why it's important: For long-term investors, a fund with high AUM but low trading volume might still be appealing because of its low expense ratio, even if it’s harder to trade. For active traders, high trading volume is more crucial to minimize transaction costs.

5. Investor Sentiment and Fund Stability
AUM can indicate investor confidence and stability. A growing AUM shows that more investors are pouring money into the fund, which can signal that the fund is stable and in demand.
Trading Volume can indicate investor sentiment and market activity. If trading volume spikes, it could indicate increased interest in the fund, either due to market volatility, news, or changes in the underlying assets.
Why it's important: A fund with large AUM and consistent trading volume is typically considered more stable and reliable, while a sudden drop in AUM or abnormal trading volume spikes could be red flags for investors.

Conclusion:
AUM helps measure the size, long-term stability, and efficiency of a fund.
Trading Volume helps gauge liquidity, market sentiment, and transaction costs.
Analyzing both together provides a comprehensive view of the fund’s liquidity, efficiency, and cost-effectiveness. High AUM with high trading volume generally indicates a well-rounded, liquid, and stable fund, whereas discrepancies between the two can suggest potential liquidity issues or speculative trading.

In [ ]:
import pandas as pd

# Sample DataFrame
data = {
    'Trader Names': ['{Trader A}, {Trader B}', '{Trader C}', '{Trader D}, {Trader E}', '{Trader A}', '{Trader B}'],
    'USD Difference': [5000, -7000, 2000, -3000, 1500],
    'Fund House': ['Fund X', 'Fund Y', 'Fund Z', 'Fund X', 'Fund Y']
}

df = pd.DataFrame(data)

# Step 1: Clean the Trader Names column
# Remove curly braces and split multiple traders into lists
df['Trader Names'] = df['Trader Names'].str.replace(r'{|}', '', regex=True)
df['Trader Names'] = df['Trader Names'].str.split(', ')

# Step 2: Expand the DataFrame so each trader has their own row
df_expanded = df.explode('Trader Names')

# Step 3: Group by Trader Names and sum the USD Difference
trader_summary = df_expanded.groupby('Trader Names', as_index=False)['USD Difference'].sum()

# Step 4: Sort to identify top positive and negative USD Differences
top_negative_traders = trader_summary.sort_values(by='USD Difference').head()
top_positive_traders = trader_summary.sort_values(by='USD Difference', ascending=False).head()

# Display results
print("Top Negative Traders:\n", top_negative_traders)
print("Top Positive Traders:\n", top_positive_traders)


In [ ]:
import pandas as pd

# Sample data
data = {
    'Trader Names': ['Trader A', 'Trader B'],
    'USD Difference': [5000, -3000],
    'Trade Date': ['31-AUG-24', '15-SEP-24']  # Dates in 'DD-MON-YY' format as strings
}

# Create DataFrame and convert 'Trade Date' to datetime
df = pd.DataFrame(data)
df['Trade Date'] = pd.to_datetime(df['Trade Date'], format='%d-%b-%y')

# Save to Excel with the date format 'DD-MON-YY'
writer = pd.ExcelWriter('trades.xlsx', engine='xlsxwriter')

# Write the dataframe to Excel
df.to_excel(writer, sheet_name='Sheet1', index=False)

# Get the xlsxwriter workbook and worksheet objects
workbook  = writer.book
worksheet = writer.sheets['Sheet1']

# Define a date format
date_format = workbook.add_format({'num_format': 'DD-MON-YY'})

# Apply the date format to the 'Trade Date' column (assuming it's in the third column, index 2)
worksheet.set_column('C:C', None, date_format)

# Save the Excel file
writer.save()


In [ ]:
import pandas as pd

# Sample dataframe with necessary columns
data = {
    'OS Cash': [1000, 2000, -1500, 3000, -4000],
    'Operation Type': ['BUY', 'SELL', 'REC', 'DEL', 'CREC'],
    'WCOFF_RATE': [0.5, 0.6, 0.7, 0.8, 0.9],
    'FX_RATE': [1.1, 1.2, 1.3, 1.4, 1.5]
}

df = pd.DataFrame(data)

# Define the operation types that correspond to the two cases
buy_rec_types = ['BUY', 'REC', 'CDEL', 'CSEL']
sell_del_types = ['SELL', 'DEL', 'CREC', 'CBUY']

# Apply the formula based on the 'Operation Type'
df['WCOF USD'] = df.apply(
    lambda row: ((-1 * row['OS Cash'] * row['WCOFF_RATE']) / 36000) * row['FX_RATE']
    if row['Operation Type'] in buy_rec_types else
    ((1 * row['OS Cash'] * row['WCOFF_RATE']) / 36000) * row['FX_RATE'],
    axis=1
)

# Show the result
print(df)
